In [ ]:
import json
from datetime import datetime
from dateutil import parser
import pandas as pd
#import calplot
import matplotlib.pyplot as plt
from urllib.parse import urlparse

In [ ]:
from pyecharts.charts import Bar, Calendar, Pie
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.globals import ThemeType
#from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

In [ ]:
with open("aw-buckets-export.json", "r") as f:
    data = json.load(f)

with open("aw-buckets-export_mac.json", "r") as f:
    mac_data = json.load(f)

In [ ]:
# Extracting Information from Windows PC watcher

app_usage = {}
dayrecord = {}
total_duration = 0.0

for event in data['buckets']['aw-watcher-window_pc']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
    duration = event['duration']
    appname = event['data']['app']
        
    if "2023" in date:
        total_duration += duration
        if appname in app_usage:
            app_usage[appname] = duration + app_usage[appname]
        else:
            app_usage[appname] = duration
    
        if appname != 'LockApp.exe':
            if date in dayrecord:
                dayrecord[date] += duration
            else:
                dayrecord[date] = duration

In [ ]:
# Getting information From mac watcher

mapp_usage = {}
mdayrecord = {}
mtotal_duration = 0.0

for event in mac_data['buckets']['aw-watcher-window_mac.local']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
    duration = event['duration']
    appname = event['data']['app']
        
    if "2023" in date:
        mtotal_duration += duration
        if appname in mapp_usage:
            mapp_usage[appname] = duration + mapp_usage[appname]
        else:
            mapp_usage[appname] = duration
    
        if appname != 'loginwindow':
            if date in mdayrecord:
                mdayrecord[date] += duration
            else:
                mdayrecord[date] = duration

In [ ]:
hours = total_duration//3600
print("Total PC Usage in hours: ", hours)
print("Total PC Usage in days: ", hours//24)

In [ ]:
dayrecord

In [ ]:
app_usage_hourly = {}
mapp_usage_hourly = {}

for k,v in app_usage.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        app_usage_hourly[k[:-4]] = usage_time

for k,v in mapp_usage.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        mapp_usage_hourly[k] = usage_time

print(app_usage_hourly)
print(mapp_usage_hourly)

In [ ]:
for k,v in mapp_usage_hourly.items():
    if k in app_usage_hourly:
        app_usage_hourly[k] += v
    else:
        app_usage_hourly[k] = v
app_usage_hourly = dict(sorted(app_usage_hourly.items(), key=lambda item: item[1], reverse=True))
print(app_usage_hourly)

In [ ]:
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))

t = [list(z) for z in zip(app_usage_hourly.keys(), app_usage_hourly.values())]
pie.add("", t)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False), title_opts=opts.TitleOpts(title="Application Usage (in hours)",  pos_left="right",))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
pie.render()
#make_snapshot(snapshot, pie.render(), "pc_app_usage.png")

# Heatmap for PC usage

In [ ]:
for k,v in mdayrecord.items():
    if k in dayrecord:
        dayrecord[k] += v
    else:
        dayrecord[k] = v

In [ ]:
len(dayrecord)

In [ ]:
dataToPlot = []

for k,v in dayrecord.items():
    date_ = datetime.strptime(k, "%Y-%m-%d")    
    if date_.year == 2023:
        dataToPlot.append([date_.strftime('%Y-%m-%d'),int(v)//3600])


In [ ]:
cal = Calendar()
cal.add("", dataToPlot, calendar_opts=opts.CalendarOpts(range_="2023"))
cal.set_global_opts(
        title_opts=opts.TitleOpts(title="PC Usage", subtitle="Hours per day"),
        visualmap_opts=opts.VisualMapOpts(
            max_=40,
            min_=0,
            orient="horizontal",
            is_piecewise=True,
            pos_top="230px",
            pos_left="100px",
        ),
    )
cal.render()

#make_snapshot(snapshot, cal.render(), 'PC_usage_heatmap.png')

# Diving into Firefox

In [ ]:
def extract_firefox_info(data):
    website_record = {}
    total_duration = 0

    for event in data['buckets']['aw-watcher-web-firefox']['events']:
        date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
        duration = event['duration']
        # website = event['data']['title']
        website = urlparse(event['data']['url']).netloc
    
        if "2023" in date:
            total_duration += duration
            if website in website_record:
                website_record[website] = duration + website_record[website]
            else:
                website_record[website] = duration
    return website_record, total_duration

wr,td = extract_firefox_info(data)
mwr, mtd = extract_firefox_info(mac_data)

In [ ]:
for k,v in wr.items():
    if k in mwr:
        mwr[k] += v
    else:
        mwr[k] = v

In [ ]:
website_hr_usage = {}

for k,v in mwr.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        website_hr_usage[k] = usage_time

sorted_website_hr_usage = dict(sorted(website_hr_usage.items(), key=lambda item: item[1], reverse=True))


In [ ]:
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
tj = [list(z) for z in zip(sorted_website_hr_usage.keys(), sorted_website_hr_usage.values())]
pie.add("", tj)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False), title_opts=opts.TitleOpts(title="Most Websites opened in firefox (in hours)",  pos_left="right",))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
pie.render()
#make_snapshot(snapshot, pie.render(), "firefox_websites.png")